In [2]:
import os
# print(os.getcwd())
os.chdir('C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol')
import pickle
import sklearn
print(sklearn.__version__)
import pandas
import numpy as np
import sys


from evaluation.metric import computeAccuracy, computeF1Score, computeAUC, computeAUCBehavior
# from sklearn.metrics import roc_auc_score

# %load_ext autoreload
# %autoreload
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from scipy.stats import uniform
# %reload_ext pipelines.alcohol.AlcoholPipeline
from pipelines.alcohol import AlcoholPipeline

0.17


In [3]:
# clf_alc_initial = pickle.load(open('./alcohol classifiers/clf_alc_UPDATED.p', 'rb'))
# clf_fpa_initial = pickle.load(open('./alcohol classifiers/clf_fpa_UPDATED.p', 'rb'))
clf_behavior_initial = pickle.load(open('./alcohol classifiers/clf_fpl_double_labeled', 'rb'))
df = pandas.read_csv("./data/alcohol_training_instances.csv")

Prepare the behavior level data

In [4]:
# appending labels based on whether tweet label contains phrase
behavior_labels = []
behavior_index_in_df = []

fpl = "'first_person_level'"

# extracting FP levels and constructing subset vector simultaneously
for i in range(len(df)):
    if fpl in df.labels[i]:
        behavior_labels.append(int(df.labels[i][df.labels[i].find(fpl) + len(fpl) + 2]))
        behavior_index_in_df.append(1)
    else:
        behavior_index_in_df.append(0)

behavior_labels = list(map(int, behavior_labels))
print(behavior_labels[:20])
# print(subset_vec_alc_fp[:10])
print('length of first person level labels: ', len(behavior_labels)) # 6357
print(behavior_index_in_df[:20])
print('length of first person level index in df: ', len(behavior_index_in_df)) # 15651
index_behavior = list(np.where(behavior_index_in_df)[0])
print(index_behavior[:15])
print('length of first person level df index: ', len(index_behavior)) # 6357
df_behavior = df.iloc[index_behavior,]
# df_fp.head
# fp_labels = np.asarray(fp_labels)

# change label 3 to label 0
behavior_labels = [0 if x == 3 else x for x in behavior_labels] 

# train test dataset split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_behavior, behavior_labels, test_size=0.2, random_state=26)

[0, 0, 2, 2, 1, 0, 0, 0, 1, 1, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0]
length of first person level labels:  6357
[0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0]
length of first person level index in df:  15651
[2, 5, 6, 7, 11, 13, 17, 18, 21, 22, 23, 24, 26, 27, 28]
length of first person level df index:  6357


In [5]:
print(len(behavior_labels))
print(len(behavior_index_in_df))
# behavior_labels[:30]
# for i in behavior_labels:
#     if i ==3:
#         print(True)

6357
15651


First we check the initial model performance before re-training. On whole behavior dataset

In [6]:
# predict in whole dataset
y_before_retrain = clf_behavior_initial.predict(df_behavior)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_before_retrain, behavior_labels)
f1_before_retrain = computeF1Score(behavior_labels, y_before_retrain, average = None)
print('F1 Score (current, looking, reflecting): ', f1_before_retrain)
auc_score_before_retrain = computeAUCBehavior(clf_behavior_initial, df_behavior, behavior_labels, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('Initial classifier AUC: ', auc_score_before_retrain)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  6357
2062 different item in whole dataset
accuracy: 67.563316%
F1 Score (current, looking, reflecting):  [0.75419892 0.63490208 0.51267375]
Initial classifier AUC:  {'current': 0.81565410674169, 'looking': 0.8359241441688562, 'reflecting': 0.806718858722408, 'micro': 0.8402825273600063, 'macro': 0.8195504247393393}


Then we check the initial model performance with training and testing

In [7]:
clf_behavior_initial.fit(X_train, y_train)
# predict in testing set.
y_behavior_initial = clf_behavior_initial.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_behavior_initial, y_test)
f1 = computeF1Score(y_test, y_behavior_initial, average = None)
print('F1 Score (current, looking, reflecting): ', f1)
auc_score_before_retrain = computeAUCBehavior(clf_behavior_initial, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('Initial classifier AUC: ', auc_score_before_retrain)


# f1 = computeF1Score(y_test, y_behavior_initial,average = None)
# print('F1 Score: ', f1)
# auc_score_initial = computeAUCBehavior(clf_behavior_initial, X_test, y_test, plot=False)
# # auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
# print('alcohol LR AUC: ', auc_score_initial)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  1272
502 different item in whole dataset
accuracy: 60.534591%
F1 Score (current, looking, reflecting):  [0.69850746 0.5730659  0.40316206]
Initial classifier AUC:  {'current': 0.7524668343015695, 'looking': 0.7975783506521129, 'reflecting': 0.7502397784756277, 'micro': 0.7919831098453383, 'macro': 0.7673463251080409}


But how Jason get so much higher performance in paper? 